# Tokenizers

このClabセッションでは、Tokenizersの世界を探索します

必要に応じて、このノートブックを無料のCPUで、またはローカル実行できます。

## リマインダー：Colabを使用するための2つの重要なプロチップ：

**Pro-tip 1:**

すべてのColabの上部には、いくつかのpipインストールがあります。これを実行すると、pipからエラーを受信する場合があります。

> gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.

これらのpip互換性エラーは安全に無視できます。そして、バージョン番号を変更することでそれらを修正しようとするのは魅力的ですが、実際に実際の問題を導入します！

**Pro-tip 2:**

Colabを実行している最中に、次のようなエラーが発生する可能性があります。

> Runtime error: CUDA is required but not available for bitsandbytes. Please consider installing [...]

これは非常に誤解を招くエラーメッセージです。パッケージのバージョンを変更しないでください。

これは、GoogleがColabランタイムを切り替えたために実際に起こります。おそらくGoogle Colabが忙しすぎたためです。解決策は次のとおりです。

1. カーネルメニュー >> ランタイムを切断して削除
2. 新鮮なメニューからColabをリロードし、メニュー >> すべての出力をクリア
3. 右上のボタンを使用して新しいT4に接続
4. 右上のメニューから「リソースを表示」を選択して、GPUがあることを確認
5. Colab内のセルを上から下へ、pipのインストールから始めて再実行

そして、すべてがうまくいくはずです - そうでなければ、私に尋ねてください！

In [1]:
# これにより、pip依存関係の競合に関する「エラー」が得られる場合は、それを無視してください！それは何にも影響しません。
!pip install -q --upgrade torch==2.5.1+cu124 torchvision==0.20.1+cu124 torchaudio==2.5.1+cu124 --index-url https://download.pytorch.org/whl/cu124
!pip install -q --upgrade transformers==4.48.3 datasets==3.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 779.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# import

# Google Colab でSecretを安全に使う
from google.colab import userdata
# Hugging Face Hub にログイン
from huggingface_hub import login
# Hugging Face の transformers の AutoTokenizer
from transformers import AutoTokenizer

# Hugging Faceにサインイン

講義でこれについては言及していませんでしたが、以前に行ったことがない場合は、Hugging Face Hubにログインする必要がある場合があります。

1. まだ行っていない場合は、https://huggingface.coで **free** Hugging Faceアカウントを作成し、右上のユーザーメニューから[設定]に移動します。次に、新しいAPIトークンを作成し、書き込み許可を与えます。  

**重要** Hugging Face APIキーを作成する場合、[書き込み]タブをクリックして、キーの読み取り/書き込み許可を必ず選択してください。そうしないと、後で問題が発生する可能性があります。

2. ここに戻って、サイドパネルの「キー」アイコンを左側に押して、新しい秘密を追加します。  
  名前フィールドに「HF_TOKEN」を入れます  
  値フィールドに実際のトークンを置きます： `hf _...`  
  「ノートブックからのアクセス	」スイッチがオンになっていることを確認してください。

3. 下のセルを実行してログインします。各Colabでこれを行う必要があります。
  SecretをColabに入力する必要なく、秘密を管理するための本当に便利な方法です。
  また、以下の行を単純に上書きするショートカットもあります。`hf_token =" hf _.... "`  
  しかし、これはベストプラクティスではありません。
  Colabを共有しないように注意する必要があるからです。
  そして、Colabについての素晴らしいことの1つは、それらを共有できることです！

In [3]:
# Hugging Face Hub にプログラムからログイン
hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

# Metaのllama 3.1へのアクセス

Fantastic Llama 3.1を使用するには、Metaは利用規約に署名する必要があります。

Hugging Faceモデルの指示ページにアクセスしてください。:
https://huggingface.co/meta-llama/Meta-Llama-3.1-8B

ページの上部には、彼らの条件に同意する方法に関する指示があります。可能であれば、Hugging Faceアカウントと同じメールを使用する必要があります。

私の経験では、承認は数分後に行われます。 3.1モデルに承認されると、3.1ファミリのモデル全体に適用されます。何らかの理由で、Metaがアクセスを承認しない場合があります。それが発生した場合は、[この](https://colab.research.google.com/drive/1deJO03YZTXUwcq2vzxWbiBhrRuI29Vo8?usp=sharing)トラブルシューティングをフォローしてください。

次のセルがあなたにエラーを与えた場合は、次のことを確認してください。  
1. Hugging Faceにログインしていますか？ `login()`を実行してみてください。APIキーを設定したか確認してください。
2. 完全な読み取りおよび書き込みアクセス許可でAPIキーを設定しましたか？
3. 上記のリンクがあるLlama3.1ページにアクセスした場合、上部近くのモデルにアクセスできることがわかりますか？

また、このトラブルシューティングコラブをセットアップして、Hugging Faceの接続性の問題を診断しようとします。  
https://colab.research.google.com/drive/1deJO03YZTXUwcq2vzxWbiBhrRuI29Vo8?usp=sharing


In [4]:
# トークナイザとトークナイズ対象文の定義
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Meta-Llama-3.1-8B', trust_remote_code=True)
text = "I am excited to show Tokenizers in action to my LLM engineers"

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [5]:
# トークナイズ（符号化）
tokens = tokenizer.encode(text)

In [6]:
# トークンIDを出力
tokens

[128000,
 40,
 1097,
 12304,
 311,
 1501,
 9857,
 12509,
 304,
 1957,
 311,
 856,
 445,
 11237,
 25175]

In [7]:
# トークンの数を確認
len(tokens)

15

In [8]:
# デコード（符号からテキストへ）
tokenizer.decode(tokens)

'<|begin_of_text|>I am excited to show Tokenizers in action to my LLM engineers'

In [9]:
# バッチデコード（複数文に対応するが、次元の関係上ココでは1単語毎になる）
# Tokenizers が Token & izers から構成されるのが興味深い。
tokenizer.batch_decode(tokens)

['<|begin_of_text|>',
 'I',
 ' am',
 ' excited',
 ' to',
 ' show',
 ' Token',
 'izers',
 ' in',
 ' action',
 ' to',
 ' my',
 ' L',
 'LM',
 ' engineers']

In [10]:
# |begin_of_text|など、追加された特殊トークン辞書
tokenizer.get_added_vocab()

{'<|begin_of_text|>': 128000,
 '<|end_of_text|>': 128001,
 '<|reserved_special_token_0|>': 128002,
 '<|reserved_special_token_1|>': 128003,
 '<|finetune_right_pad_id|>': 128004,
 '<|reserved_special_token_2|>': 128005,
 '<|start_header_id|>': 128006,
 '<|end_header_id|>': 128007,
 '<|eom_id|>': 128008,
 '<|eot_id|>': 128009,
 '<|python_tag|>': 128010,
 '<|reserved_special_token_3|>': 128011,
 '<|reserved_special_token_4|>': 128012,
 '<|reserved_special_token_5|>': 128013,
 '<|reserved_special_token_6|>': 128014,
 '<|reserved_special_token_7|>': 128015,
 '<|reserved_special_token_8|>': 128016,
 '<|reserved_special_token_9|>': 128017,
 '<|reserved_special_token_10|>': 128018,
 '<|reserved_special_token_11|>': 128019,
 '<|reserved_special_token_12|>': 128020,
 '<|reserved_special_token_13|>': 128021,
 '<|reserved_special_token_14|>': 128022,
 '<|reserved_special_token_15|>': 128023,
 '<|reserved_special_token_16|>': 128024,
 '<|reserved_special_token_17|>': 128025,
 '<|reserved_special_to

# モデルのバリエーションを指示します

多くのモデルには、チャットで使用するためにトレーニングされているバリアントがあり通常、末尾が「-Instruct」でラベル付けされています。  
これらは、システム、ユーザー、アシスタントプロンプトを含む特定の形式でプロンプトを期待するように訓練されています。  

馴染みのあるメッセージリスト形式から、このモデルの適切な入力プロンプトに変換されるユーティリティメソッド `Apply_Chat_Template`があります。

In [11]:
# ベースモデルを Supervised Fine-Tuning（SFT） と RLHF（強化学習＋人間のフィードバック） で調整したプロンプトバージョン
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Meta-Llama-3.1-8B-Instruct', trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [12]:
# Chat形式の入力メッセージ（messages）を、
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "Tell a light-hearted joke for a room of Data Scientists"}
  ]

# 言語モデルに送るための1本のプロンプト文字列に変換
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a helpful assistant<|eot_id|><|start_header_id|>user<|end_header_id|>

Tell a light-hearted joke for a room of Data Scientists<|eot_id|><|start_header_id|>assistant<|end_header_id|>




# 新しいモデルを試してみます
3つのモデルを使用

In [13]:
# MicrosoftのPhi3
PHI3_MODEL_NAME = "microsoft/Phi-3-mini-4k-instruct"
# Alibaba CloudのQWEN2
QWEN2_MODEL_NAME = "Qwen/Qwen2-7B-Instruct"
# bigcode（serviceNow + huggingface + nvidia）のstarcoder2）
STARCODER2_MODEL_NAME = "bigcode/starcoder2-3b"

## MicrosoftのPhi3

In [14]:
phi3_tokenizer = AutoTokenizer.from_pretrained(PHI3_MODEL_NAME)
text = "I am excited to show Tokenizers in action to my LLM engineers"

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

In [15]:
# encode

# MetaのLlama 3.1
tokens_Llama3 =tokenizer.encode(text)
print(tokens_Llama3)

print("----------------------------------------------------------------------------------------------------")

# MicrosoftのPhi3
tokens_phi3 = phi3_tokenizer.encode(text)
print(tokens_phi3)

[128000, 40, 1097, 12304, 311, 1501, 9857, 12509, 304, 1957, 311, 856, 445, 11237, 25175]
----------------------------------------------------------------------------------------------------
[306, 626, 24173, 304, 1510, 25159, 19427, 297, 3158, 304, 590, 365, 26369, 6012, 414]


In [16]:
# batch_decode
print(phi3_tokenizer.batch_decode(tokens_phi3))

['I', 'am', 'excited', 'to', 'show', 'Token', 'izers', 'in', 'action', 'to', 'my', 'L', 'LM', 'engine', 'ers']


In [17]:
#apply_chat_template

# MetaのLlama 3.1
print(tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True))

print("----------------------------------------------------------------------------------------------------")

# MicrosoftのPhi3
print(phi3_tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a helpful assistant<|eot_id|><|start_header_id|>user<|end_header_id|>

Tell a light-hearted joke for a room of Data Scientists<|eot_id|><|start_header_id|>assistant<|end_header_id|>


----------------------------------------------------------------------------------------------------
<|system|>
You are a helpful assistant<|end|>
<|user|>
Tell a light-hearted joke for a room of Data Scientists<|end|>
<|assistant|>



## Alibaba CloudのQWEN2

In [18]:
qwen2_tokenizer = AutoTokenizer.from_pretrained(QWEN2_MODEL_NAME)

# MetaのLlama 3.1
print(tokens_Llama3)

print("----------------------------------------------------------------------------------------------------")

# MicrosoftのPhi3
print(tokens_phi3)

print("----------------------------------------------------------------------------------------------------")

# Alibaba CloudのQWEN2
tokens_qwen2 = qwen2_tokenizer.encode(text)
print(tokens)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

[128000, 40, 1097, 12304, 311, 1501, 9857, 12509, 304, 1957, 311, 856, 445, 11237, 25175]
----------------------------------------------------------------------------------------------------
[306, 626, 24173, 304, 1510, 25159, 19427, 297, 3158, 304, 590, 365, 26369, 6012, 414]
----------------------------------------------------------------------------------------------------
[128000, 40, 1097, 12304, 311, 1501, 9857, 12509, 304, 1957, 311, 856, 445, 11237, 25175]


In [19]:
# MetaのLlama 3.1
print(tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True))

print("----------------------------------------------------------------------------------------------------")

# MicrosoftのPhi3
print(phi3_tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True))

print("----------------------------------------------------------------------------------------------------")

# Alibaba CloudのQWEN2
print(qwen2_tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a helpful assistant<|eot_id|><|start_header_id|>user<|end_header_id|>

Tell a light-hearted joke for a room of Data Scientists<|eot_id|><|start_header_id|>assistant<|end_header_id|>


----------------------------------------------------------------------------------------------------
<|system|>
You are a helpful assistant<|end|>
<|user|>
Tell a light-hearted joke for a room of Data Scientists<|end|>
<|assistant|>

----------------------------------------------------------------------------------------------------
<|im_start|>system
You are a helpful assistant<|im_end|>
<|im_start|>user
Tell a light-hearted joke for a room of Data Scientists<|im_end|>
<|im_start|>assistant



## bigcode（serviceNow + huggingface + nvidia）のstarcoder2）

In [20]:
# 軽量ながら高性能なCode LLM
starcoder2_tokenizer = AutoTokenizer.from_pretrained(STARCODER2_MODEL_NAME, trust_remote_code=True)
code = """
def hello_world(person):
  print("こんにちは", person)
"""
tokens = starcoder2_tokenizer.encode(code)
for token in tokens:
  print(f"{token}={starcoder2_tokenizer.decode(token)}")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

222=

610=def
17966= hello
100=_
5879=world
45=(
6427=person
731=):
353=
 
1489= print
459=("
7695=こ
16282=ん
3885=に
28666=ち
4996=は
411=",
4944= person
46=)
222=

